# Devoir Python

Vous devez rendre votre devoir sur GitHub.
Vous avez le droit a tout vos documents et a internet

1. votre depot doit etre privé
2. vous devez inviter comme colaborateur votre chargé de TD/TP
3. Seul le dernier commit avant la fin de la séance sera corrigé.


Ex 1: Integrale de Romberg

Ecrire une fonction integ_romberg(f, a, b, epsilon=1e-6) permettant de calculer l’intégrale numérique de la fonction f entre les bornes a et b avec une précision epsilon selon la méthode de Romberg (https://fr.wikipedia.org/wiki/M%C3%A9thode_de_Romberg).

Il s’agit d’une méthode qui permet d’améliorer les méthodes usuelles de calcul numérique des intégrales, comme la méthode des Trapèzes.
On montre qu’en combinant judicieusement les valeurs obtenues par la méthode des Trapèzes pour différentes subdivisons de l’intervalle d’intégration, on augmente l’ordre de convergence (sa vitesse de convergence). 


In [38]:
import numpy as np

def integ_romberg(f, a, b, epsilon=1e-6):
    n = 10 #nombre max de niveaux de récurrence
    resultat = np.array( [[0] * (n+1)] * (n+1), float )
    h = b - a
    resultat[0,0] = 0.5*h*(f(a)+f(b))

    kMax = 1
    for i in range( 1, n + 1 ):
        h = 0.5 * h

        T = 0.0
        kMax = 2 * kMax
        for k in range( 1, kMax, 2 ):
            T = T + f(a+k*h)

        # La méthode des trapèzes :
        resultat[i,0] = 0.5 * resultat[i-1,0]+T*h
        m = 1
        for j in range(1,i+1):
            m = 4 * m
            resultat[i,j] = resultat[i,j-1] + (resultat[i,j-1]-resultat[i-1,j-1])/(m-1)
        
        if (abs(resultat[i,j] - resultat[i,j-1]) < epsilon):        #Si on a atteint la précision demandée, on arrête
            break
    return resultat[i,j]

def f1(x):
   f1 = x**5
   return f1

def f2(x):
   f2 = np.exp(-x*x)
   return f2

def f3(x):
   tau   = 1.0e-8
   f3    = np.where(np.abs(x)<tau,1.0,np.sin(x)/x)
   return f3

print(integ_romberg(f1,0,10,1.0e-12))

166666.66666666666


Ex 2: Équation d’état de l’eau à partir de la dynamique moléculaire

Afin de modéliser les planètes de type Jupiter, Saturne, ou même des exo-planètes très massives (dites « super-Jupiters »), la connaissance de l’équation d’état des composants est nécessaire. Ces équations d’état doivent être valables jusqu’à plusieurs centaines de méga-bar ; autrement dit, celles-ci ne sont en aucun cas accessibles expérimentalement. On peut cependant obtenir une équation d’état numériquement à partir d’une dynamique moléculaire.

Le principe est le suivant : on place dans une boite un certain nombre de particules régies par les équations microscopiques (Newton par exemple, ou même par des équations prenant en considération la mécanique quantique) puis on laisse celles-ci évoluer dans la boite ; on calcule à chaque pas de temps l’énergie interne à partir des intéractions électrostatiques et la pression à partir du tenseur des contraintes. On obtient en sortie l’évolution du système pour une densité fixée (par le choix de taille de la boite) et une température fixée (par un algorithme de thermostat que nous ne détaillerons pas ici).

On se propose d’analyser quelques fichiers de sortie de tels calculs pour l’équation d’état de l’eau à très haute pression. Les fichiers de sortie sont disponibles ici; leur nom indique les conditions thermodynamiques correspondant au fichier, p.ex. 6000K_30gcc.out pour T=6000
K et ρ=30 gcc. Le but est, pour chaque condition température-densité, d’extraire l’évolution de l’énergie et de la pression au cours du temps, puis d’en extraire la valeur moyenne ainsi que les fluctuations. Il arrive souvent que l’état initial choisi pour le système ne corresponde pas à son état d’équilibre, et qu’il faille donc « jeter » les quelques pas de temps en début de simulation qui correspondent à cette relaxation du système. Pour savoir combien de temps prend cette relaxation, il sera utile de tracer l’évolution au cours du temps de la pression et l’énergie pour quelques simulations. Une fois l’équation d’état P(ρ,T) et E(ρ,T) extraite, on pourra tracer le réseau d’isothermes.

In [73]:
import numpy as np
from matplotlib import pyplot as plt 

#liste des fichiers outputs
from os import listdir
from os.path import isfile, join
mypath="outputs/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

plots = [] #liste des graphs
for i in onlyfiles:
    print(i)
    with open(mypath + i) as out: #on ouvre le fichier out
        lines = list(out)
        energy = {"y":np.array( [0] * (len(lines)-1), float ), "x":np.arange(1,len(lines)) }
        for i in range(len(lines[1:])):
            pE = lines[i+1].split(" ")[0]
            if '\t' in pE:
                pE = pE.split('\t')[0]
            energy["y"][i] = float(pE)
        plots.append(energy)
        print(len(lines))
        
plt.title("Matplotlib demo") 
plt.xlabel("pas") 
plt.ylabel("enegie") 
plt.plot(plots[8]["x"],plots[8]["y"]) 
plt.show()

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'outputs/'

Ex 3: Le problème du voyageur de commerce

Le problème du voyageur de commerce est un problème d’optimisation consistant à déterminer le plus court chemin reliant un ensemble de destinations. Il n’existe pas d’algorithme donnant la solution optimale en un temps raisonnable (problème NP-complet), mais l’on peut chercher à déterminer des solutions approchées. On va se placer ici dans le cas d’un livreur devant desservir une seule fois chacune desndestinations d’une ville américaine où les rues sont agencées en réseau carré. 

On utilise la « distance deManhattan »  entre deux points 𝐴(𝑥𝐴,𝑦𝐴) et 𝐵(𝑥𝐵,𝑦𝐵) : 𝑑(𝐴,𝐵) =|𝑥𝐵−𝑥𝐴|+|𝑦𝐵−𝑦𝐴|.

En outre, on se place dans le cas où les coordonnées des destinations sont entières, comprises entre 0 (inclus) et TAILLE = 50 (exclus). Deux destinations peuvent éventuellement avoir les mêmes coordonnées. Les instructions suivantes doivent permettre de définir les classes nécessaires (Ville et Trajet) et de développer un algorithme approché (heuristiques) : l’algorithme du plus proche voisin. 

Seules la librairie standard et la librairie numpy sont utilisables si nécessaire. Implementer les classes et methodes suivante:


Classe Ville:
* __init__(): initialisation d’une ville sans destination.
* aleatoire(n): création de n destinations aléatoires.
* nb_trajet(): retourne le nombre total (entier) de trajets :(𝑛−1)!/2(utilisermath.factorial()).
* distance(i, j): retourne la distance (Manhattan) entre les deux destinations de numéro i et j


Classe Trajet:
* __init__(ville, etapes=None): initialisation sur une ville. Si la liste etapes n’est pas spécifiée, le trajet par défaut est celui suivant les destinations de ville.
* longueur(): retourne la longueur totale du trajetbouclé(i.e. revenant à son point de départ).


Plus proche voisin:
* Ville.plus_proche(i, exclus=[]): retourne la destination la plus proche de la destinationi(au sens de Ville.distance()), hors les destinations de la liste exclus
* Ville.trajet_voisins(depart=0): retourne un Trajet déterminé selon l’heuristique des plus proches voisins (i.e. l’étape suivante est la destination la plus proche hors les destinations déjà visitées) en partant de l’étape initiale depart

Optimisation:
* Proposer un algorithme qui propose une meilleur alternative au "plus proche voisin".

Interface:
* Ville.figure(trajet=None): Afficher le plande la ville et le trajet obtenue (utiliser matplotlib.step()pour des trajets de type « Manhattan »)

In [71]:
import numpy as np
import math
TAILLE = 50

class Ville(object):

    def __init__(self):
        self.destinations = np.array([]).reshape(-1, 2) #reshape le tableau : 2 colonnes par ligne

    def aleatoire(self, n):
        self.destinations = np.random.randint(TAILLE, size=(n, 2))#rempli le tableau destination de valeur aléatoire

    def nb_trajets(self):
        nbdest = len(self.destinations)
        if nbdest > 2:
            return int(math.factorial(nbdest - 1) / 2) #retourne le nombre total (entier) de trajets
        elif nbdest > 0:
            return 1
        else:
            return 0

    def distance(self, i, j):

        return np.abs(self.destinations[i] - self.destinations[j]).sum() #calcul distance manathan
    
    def plus_proche(self, i, exclus=[]):

        voisins=[]
        for j in range(len(self.destinations)):
            if(j not in exclus and j!=i): # on verifie que la dest n'est pas exclu ou n'est pas égale à la dest d'origine
                voisins = [j] #ajoute au tableau des voisin
        for j in voisins:
            distances = [self.distance(i, j)] #calcul la distance des voisins
        return voisins[np.argmin(distances)]#retourne la destination avec la distance minimum
    
    def trajet_voisins(self, depart=0):
        nbdest = len(self.destinations)
        if depart is None:     
            for i in range(nbdest): # itère sur chaque destination
                trajets = trajets.append(self.trajet_voisins(depart=i)) #ajoute chaque destination au trajet
                for t in (trajets) :
                    longueurs = longueurs.append(t.longueur()) #calcul la longeur des trajets
            return trajets[np.argmin(longueurs)] #rappelle la fonction avec le bon départ
        else:                  
            etapes = [depart]
            while len(etapes) < nbdest: #tant qu'on n'est pas arrivé à la destination finale
                i = etapes[-1]
                j = self.plus_proche(i, exclus=etapes[:-1]) #calcul de la destination la plus proche, rajout de la precedente destination a exclu
                etapes.append(j)
            return Trajet(self, etapes) #crée le trajet

class Trajet(object):

    def __init__(self, ville, etapes=None):
        self.ville = ville
        if etapes is None:                     
            self.etapes = np.arange(len(self.ville.destinations)) # quand etape est none le trajet par défaut est le parcours des destinations de la ville
        else:
            self.etapes = np.array(etapes)

    def longueur(self):
        long=0
        for i in range(len(self.etapes)-1):
            long += self.ville.distance(self.etapes[i], self.etapes[i+1]) #ajoute la distance entre chaque étape
        long+= self.ville.distance(self.etapes[-1], self.etapes[0]) # retour au point de départ
        return long             


v= Ville()
v.aleatoire(5)
print(v.destinations)
print(v.nb_trajets())
print(v.distance(0,4))
print(v.plus_proche(1,{1}))
                       
t= Trajet(v)
print(t.longueur())
t2=v.trajet_voisins([1, 0, 2, 3])
print(t2.longueur())
                       


[[46 29]
 [48  2]
 [ 4 38]
 [32 27]
 [24 15]]
12
36
4
204
425
